In [1]:
56

56

In [1]:
import pandas as pd


#df = pd.read_parquet('artifacts/train.parquet')

In [3]:
df

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
1837,Out of SF,Monday,10.6,15.8,1.0,3,12,2018,4
16930,Taraval,Monday,10.5,8.3,0.0,30,12,2019,42
6824,Taraval,Monday,17.6,16.6,0.0,21,6,2021,24
5665,Bayview,Sunday,15.3,6.1,0.0,17,1,2021,1
19722,Tenderloin,Tuesday,13.0,10.1,0.0,2,3,2021,21
...,...,...,...,...,...,...,...,...,...
11284,Out of SF,Friday,12.7,7.9,0.0,28,10,2022,1
11964,Central,Monday,11.1,11.2,0.0,23,1,2023,1
5390,Tenderloin,Wednesday,16.2,12.6,0.0,18,11,2020,1
860,Richmond,Monday,15.2,18.7,0.0,4,6,2018,2


In [8]:
train_array = pd.read_parquet('artifacts/train.parquet')
test_array = pd.read_parquet('artifacts/test.parquet')

In [9]:
            X_train,y_train,X_test,y_test=(
                train_array[:,:-1],
                train_array[:,-1],
                test_array[:,:-1],
                test_array[:,-1]
            )

InvalidIndexError: (slice(None, None, None), slice(None, -1, None))

In [12]:
train_array

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
1837,Out of SF,Monday,10.6,15.8,1.0,3,12,2018,4
16930,Taraval,Monday,10.5,8.3,0.0,30,12,2019,42
6824,Taraval,Monday,17.6,16.6,0.0,21,6,2021,24
5665,Bayview,Sunday,15.3,6.1,0.0,17,1,2021,1
19722,Tenderloin,Tuesday,13.0,10.1,0.0,2,3,2021,21
...,...,...,...,...,...,...,...,...,...
11284,Out of SF,Friday,12.7,7.9,0.0,28,10,2022,1
11964,Central,Monday,11.1,11.2,0.0,23,1,2023,1
5390,Tenderloin,Wednesday,16.2,12.6,0.0,18,11,2020,1
860,Richmond,Monday,15.2,18.7,0.0,4,6,2018,2


In [13]:
    obj=DataIngestion()
    train_data,test_data=obj.initiate_data_ingestion()

NameError: name 'DataIngestion' is not defined

In [51]:
import sys
import os
from src.exception import CustomException
from src.logger import logging
from src.utils import read_data_from_mongodb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from dataclasses import dataclass

from src.components.data_transformation import DataTransformation
from src.components.data_transformation import DataTransformationConfig

from src.components.model_trainer import ModelTrainerConfig
from src.components.model_trainer import ModelTrainer

@dataclass
class DataIngestionConfig:
    train_data_path : str = os.path.join('artifacts','train.parquet')
    test_data_path : str = os.path.join('artifacts','test.parquet')
    raw_data_path : str = os.path.join('artifacts','data.parquet')


class DataIngestion:
    def __init__(self):
        self.ingestion_config = DataIngestionConfig()

    def initiate_data_ingestion(self):
        logging.info('Entered the data ingestion method')

        try:
            #df = pd.read_parquet('data/data_for_model_prep.csv')
            df = read_data_from_mongodb(database_name='PoliceIncidents',table_name='data_model_training')
            logging.info('Read the dataset from mongo db')

            os.makedirs(os.path.dirname(self.ingestion_config.train_data_path),exist_ok=True)
            df.to_parquet(self.ingestion_config.raw_data_path)

            logging.info('Train test split initiated')
            train_set, test_set = train_test_split(df,test_size=0.2,random_state=42)

            train_set.to_parquet(self.ingestion_config.train_data_path)
            test_set.to_parquet(self.ingestion_config.test_data_path)

            logging.info('Data ingestion completed')

            return (
                self.ingestion_config.train_data_path,
                self.ingestion_config.test_data_path

            )
        except Exception as e:
            raise CustomException(e,sys)
            

In [15]:
    obj=DataIngestion()
    train_data,test_data=obj.initiate_data_ingestion()

In [16]:
train_data

'artifacts/train.parquet'

In [20]:

    data_transformation=DataTransformation()
    train_array,test_array,_=data_transformation.initiate_data_transformation(train_data,test_data)

In [21]:
            X_train,y_train,X_test,y_test=(
                train_array[:,:-1],
                train_array[:,-1],
                test_array[:,:-1],
                test_array[:,-1]
            )

In [24]:
            models = {
                "Random Forest": RandomForestRegressor(),
                "Decision Tree": DecisionTreeRegressor(),
                "Gradient Boosting": GradientBoostingRegressor(),
                "Linear Regression": LinearRegression(),
                "XGBRegressor": XGBRegressor(),
                "CatBoosting Regressor": CatBoostRegressor(verbose=False),
                "AdaBoost Regressor": AdaBoostRegressor(),
            }
            params={
                "Decision Tree": {
                    'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                    # 'splitter':['best','random'],
                    # 'max_features':['sqrt','log2'],
                },
                "Random Forest":{
                    # 'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                 
                    # 'max_features':['sqrt','log2',None],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Gradient Boosting":{
                    # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
                    'learning_rate':[.1,.01,.05,.001],
                    'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
                    # 'criterion':['squared_error', 'friedman_mse'],
                    # 'max_features':['auto','sqrt','log2'],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Linear Regression":{},
                "XGBRegressor":{
                    'learning_rate':[.1,.01,.05,.001],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "CatBoosting Regressor":{
                    'depth': [6,8,10],
                    'learning_rate': [0.01, 0.05, 0.1],
                    'iterations': [30, 50, 100]
                },
                "AdaBoost Regressor":{
                    'learning_rate':[.1,.01,0.5,.001],
                    # 'loss':['linear','square','exponential'],
                    'n_estimators': [8,16,32,64,128,256]
                }
                
            }


In [25]:

            model_report:dict=evaluate_models(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,
                                             models=models,param=params)

In [26]:
model_report

{'Random Forest': 0.20459357935246247,
 'Decision Tree': -0.5297488664339993,
 'Gradient Boosting': 0.21458136893617585,
 'Linear Regression': 0.13636311822499125,
 'XGBRegressor': 0.22853432375418514,
 'CatBoosting Regressor': 0.23616838236629312,
 'AdaBoost Regressor': 0.10737619576557922}

In [27]:
model = RandomForestRegressor()

In [28]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [29]:
model.score(X_train, y_train)

0.8866434569157364

In [30]:
ypred = model.predict(X_test)

In [40]:
r2_score(y_test, ypred)

0.20195621063282065

In [42]:
model.score(X_test,y_test)

0.20195621063282065

In [43]:
model.score(X_train,y_train)

0.8866434569157364

In [46]:
import sys
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


from src.exception import CustomException
from src.logger import logging

from sodapy import Socrata
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily


##   Police Department Incident Reports: 2018 to Present ( data.sfgov.org )
def fetch_data_incidents(limit : int):
    '''
    This function fetches data from the San Francisco Data API, 
    limit -> no of records fetched
    '''
    # get client
    client = Socrata("data.sfgov.org", None)

    #get data from the API
    results = client.get("wg3w-h783", limit=limit)

    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)
    columns_to_select = ['incident_date', 'incident_year', 'incident_day_of_week', 'incident_category',
                     'incident_description', 'resolution', 'police_district', 'latitude', 'longitude']
    results_df = results_df[columns_to_select]

    return results_df

In [53]:
df = fetch_data_incidents(limit = 300000)

In [48]:
df = read_data_from_mongodb(database_name='PoliceIncidents',table_name='data_model_training')

In [52]:
df

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
0,Bayview,Monday,9.3,4.3,0.0,1,1,2018,2
1,Central,Monday,9.3,4.3,0.0,1,1,2018,4
2,Ingleside,Monday,9.3,4.3,0.0,1,1,2018,2
3,Mission,Monday,9.3,4.3,0.0,1,1,2018,4
4,Northern,Monday,9.3,4.3,0.0,1,1,2018,2
...,...,...,...,...,...,...,...,...,...
14410,Park,Tuesday,16.9,12.1,0.0,25,4,2023,3
14411,Richmond,Tuesday,16.9,12.1,0.0,25,4,2023,11
14412,Southern,Tuesday,16.9,12.1,0.0,25,4,2023,28
14413,Taraval,Tuesday,16.9,12.1,0.0,25,4,2023,5


In [50]:
df.iloc[0:20]

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
0,Bayview,Monday,9.3,4.3,0.0,1,1,2018,2
1,Central,Monday,9.3,4.3,0.0,1,1,2018,4
2,Ingleside,Monday,9.3,4.3,0.0,1,1,2018,2
3,Mission,Monday,9.3,4.3,0.0,1,1,2018,4
4,Northern,Monday,9.3,4.3,0.0,1,1,2018,2
5,Out of SF,Monday,9.3,4.3,0.0,1,1,2018,7
6,Park,Monday,9.3,4.3,0.0,1,1,2018,1
7,Richmond,Monday,9.3,4.3,0.0,1,1,2018,4
8,Southern,Monday,9.3,4.3,0.0,1,1,2018,5
9,Taraval,Monday,9.3,4.3,0.0,1,1,2018,2


In [60]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.sfgov.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.sfgov.org,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("wg3w-h783", limit=10000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [61]:
d1 = results_df.copy()

In [62]:
d1['incident_datetime'].min()

'2018-01-01T19:00:00.000'

In [63]:
d1['incident_datetime'].max()

'2023-04-11T18:42:00.000'

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.sfgov.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata("data.sfgov.org",
                  app_token='T6GNOI2SBaeRzeggL3AdluYVw',
                  username="jeenapiyush101@gmail.com",
                  password="MuB$rAJU2XKwsq7")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("wg3w-h783", limit=800000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [2]:
results_df.shape

(722608, 34)

In [2]:
df= pd.read_parquet('artifacts/data.parquet')

In [3]:
df

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
0,Bayview,Monday,9.3,4.3,0.0,1,1,2018,2
1,Central,Monday,9.3,4.3,0.0,1,1,2018,4
2,Ingleside,Monday,9.3,4.3,0.0,1,1,2018,2
3,Mission,Monday,9.3,4.3,0.0,1,1,2018,4
4,Northern,Monday,9.3,4.3,0.0,1,1,2018,2
...,...,...,...,...,...,...,...,...,...
21346,Bayview,Sunday,8.7,20.2,1.0,26,2,2023,39
21347,Park,Sunday,8.7,20.2,1.0,26,2,2023,7
21348,Tenderloin,Sunday,8.7,20.2,1.0,26,2,2023,43
21349,Richmond,Tuesday,8.0,22.7,1.0,28,2,2023,16


In [6]:
df['no_of_crimes'].min()

1

In [7]:
df['no_of_crimes'].max()

153

In [8]:
df

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
0,Bayview,Monday,9.3,4.3,0.0,1,1,2018,2
1,Central,Monday,9.3,4.3,0.0,1,1,2018,4
2,Ingleside,Monday,9.3,4.3,0.0,1,1,2018,2
3,Mission,Monday,9.3,4.3,0.0,1,1,2018,4
4,Northern,Monday,9.3,4.3,0.0,1,1,2018,2
...,...,...,...,...,...,...,...,...,...
21346,Bayview,Sunday,8.7,20.2,1.0,26,2,2023,39
21347,Park,Sunday,8.7,20.2,1.0,26,2,2023,7
21348,Tenderloin,Sunday,8.7,20.2,1.0,26,2,2023,43
21349,Richmond,Tuesday,8.0,22.7,1.0,28,2,2023,16


In [9]:
train = pd.read_parquet('artifacts/train.parquet')

In [10]:
train

,police_district,incident_day_of_week,avg_temperature,wind_speed,game_day,day,month,year,no_of_crimes
1837,Out of SF,Saturday,14.4,9.4,1.0,24,11,2018,2
16930,Ingleside,Tuesday,11.2,20.5,0.0,12,3,2019,23
6824,Ingleside,Saturday,13.7,16.2,0.0,22,5,2021,5
5665,Out of SF,Tuesday,10.6,5.8,0.0,1,12,2020,1
19722,Mission,Monday,18.9,15.1,0.0,3,8,2020,37
...,...,...,...,...,...,...,...,...,...
11284,Central,Saturday,14.4,24.5,0.0,2,7,2022,54
11964,Tenderloin,Thursday,18.2,14.4,0.0,1,9,2022,49
5390,Richmond,Wednesday,20.9,7.2,0.0,30,9,2020,3
860,Tenderloin,Tuesday,16.7,14.4,0.0,29,5,2018,1


In [11]:
df = pd.read_parquet('data/Police_Department_Incident_Reports_2018_1_1_to_today.parquet')

In [12]:
df

,incident_date,incident_year,incident_day_of_week,incident_category,incident_description,resolution,police_district,latitude,longitude
0,2023-03-13T00:00:00.000,2023,Monday,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,None,None
1,2023-03-01T00:00:00.000,2023,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Mission,None,None
2,2023-03-13T00:00:00.000,2023,Monday,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,None,None
3,2023-03-13T00:00:00.000,2023,Monday,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,None,None
4,2023-03-14T00:00:00.000,2023,Tuesday,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,None,None
...,...,...,...,...,...,...,...,...,...
199995,2022-09-22T00:00:00.000,2022,Thursday,Non-Criminal,Mental Health Detention,Open or Active,Bayview,37.72829372549084,-122.38755070874484
199996,2022-09-11T00:00:00.000,2022,Sunday,Larceny Theft,"Theft, Other Property, $50-$200",Open or Active,Southern,None,None
199997,2022-09-01T00:00:00.000,2022,Thursday,Traffic Collision,Traffic Collision,Cite or Arrest Adult,Northern,37.780576217006235,-122.42025546027398
199998,2022-10-27T00:00:00.000,2022,Thursday,Assault,"Battery, former Spouse or Dating Relationship",Open or Active,Southern,37.77622135530407,-122.41160611386445
